You have to work on the [University dataset](https://drive.google.com/drive/folders/1Hs3nRtK_F3h8eg59B4-TD1DEua6g8Klv?usp=sharing). It contains three different university rankings:
*  The Times Higher Education World University Ranking, shortly *Times*,
*  the Academic Ranking of World Universities, shortly *Shanghai*,
*  the Center for World University Rankings, shortly *cwur*.

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).

## Elenco variabili usate:
##### - timesData = dataframe "timesData.csv" contenente le colonne ['world_rank', 'university_name', 'country', 'teaching', 'international','research', 'citations', 'income', 'total_score', 'num_students','student_staff_ratio', 'international_students', 'female_male_ratio','year']
##### - shanghaiData = dataframe "shanghaiData" contenente le colonne ['world_rank', 'university_name', 'national_rank', 'total_score','alumni', 'award', 'hici', 'ns', 'pub', 'pcp', 'year']
##### - cwurData = dataframe "cwurData" contenente le colonne ['world_rank', 'institution', 'country', 'national_rank','quality_of_education', 'alumni_employment', 'quality_of_faculty','publications', 'influence', 'citations', 'broad_impact', 'patents','score', 'year']
##### - timesDataIncomeNotNull = dataframe timesData rimuovendo le row con income mancante
##### - timesDataMaleFemaleNotNull = dataframe timesData rimuovendo le row con conteggio studenti o ratio M/F mancante
##### - universityListUnique[Times/Shangai/Cwur] = lista dei nomi delle università senza ripetizioni prese dai dataset di input
##### - mostRecent = timesDataIncomeNotNull dataframe ma mantenendo solo le date più recenti di ogni università
##### - lastRecent = timesDataIncomeNotNull dataframe ma mantendo solo le date meno recenti di ogni università
##### - incomeDifference = dataframe contenente per ogni università di timesData la differenza di income tra l'anno più recente e quello meno recente
##### - maleFemaleStudentsCount = dataframe contente il conteggio totale di studenti con proporzione m/f raggruppati per country
##### - datasetRanking = dataset ottenuto con una full outer join dei tra dataset di input mostrando il ranking più recente per università
##### - mostRecent[Times/Shanghai/Cwur] = datasets contenenti i dati più recenti di ogni università presi dai tre diversi dataset di input

### Analisi e pulizia dei dataset:


In [1]:
import pandas as pd
import re
import numpy as np

#importazione dei dataset
timesData = pd.read_csv("timesData.csv")
shanghaiData = pd.read_csv("shanghaiData.csv")
cwurData = pd.read_csv("cwurData.csv")

#controllo che la colonna income di timesData non contenga valori nulli o il carattere "-", altrimenti elimino la riga
timesDataIncomeNotNull = timesData.drop(timesData.index[timesData["income"] == '-'])
timesDataIncomeNotNull = timesDataIncomeNotNull.dropna(subset=["income"])
#trasformo la colonna income in double
try:
    timesDataIncomeNotNull = timesDataIncomeNotNull.astype({'income': 'double'})
except:
    print("Non è stato possibile convertire la colonna income")


#controllo che le colonne num_students e male_female_ratio non siano nulle, altrimenti elimino la riga
timesDataMaleFemaleNotNull = timesData.dropna(subset=["num_students","female_male_ratio"])
timesDataMaleFemaleNotNull = timesDataMaleFemaleNotNull.drop(timesDataMaleFemaleNotNull.index[timesDataMaleFemaleNotNull["female_male_ratio"] == '-'])
timesDataMaleFemaleNotNull["num_students"] = timesDataMaleFemaleNotNull["num_students"].str.replace(",","")
try:
    timesDataMaleFemaleNotNull = timesDataMaleFemaleNotNull.astype({'num_students': 'int32'})
except:
    print("Non è stato possibile convertire la colonna num_students")

### 1. For each university, extract from the *times* dataset the most recent and the least recent data, obtaining two separate dataframes <font color='green'>Done</font> 

In [2]:
#ottengo dal dataset "timesData" la lista con i nomi di tutte le università prese una sola volta e inserisco in "universityListUnique"
universityListUniqueTimes = []
universityListUniqueTimes = timesDataIncomeNotNull["university_name"].unique()
mostRecent = pd.DataFrame(columns = timesDataIncomeNotNull.columns)
lastRecent = pd.DataFrame(columns = timesDataIncomeNotNull.columns)
#per ogni università inserisco nei rispettivi dataset solo le riga con la data meno recente e più recente 
for university in universityListUniqueTimes:
    mostRecent = mostRecent.append((timesDataIncomeNotNull[timesDataIncomeNotNull["university_name"] == university].sort_values("year")).tail(1))
    lastRecent = lastRecent.append((timesDataIncomeNotNull[timesDataIncomeNotNull["university_name"] == university].sort_values("year")).head(1))

### 2. For each university, compute the improvement in `income` between the least recent and the most recent data points <font color='green'>Done</font> 

In [3]:
#popolo il dataset incomeDifference calcolando per ogni università la differenza di income tra il dato più recente e meno recente
incomeDifference = pd.DataFrame(columns = ["University", "Income_difference"])
for university in universityListUniqueTimes:
    differenceCount = round(float((mostRecent[mostRecent["university_name"] == university]["income"])) - float((lastRecent[lastRecent["university_name"] == university]["income"])),2)
    incomeDifference = incomeDifference.append(pd.Series({'University':university, 'Income_difference':differenceCount }), ignore_index=True)
incomeDifference

,University,Income_difference
0,Harvard University,10.7
1,California Institute of Technology,14.1
2,Massachusetts Institute of Technology,7.9
3,Stanford University,-1.0
4,University of Cambridge,-2.0
...,...,...
772,Xidian University,0.0
773,Yeungnam University,0.0
774,Yıldız Technical University,0.0
775,Yokohama City University,0.0


### 3. Find the university with the largest increase computed in the previous point <font color='green'>Done</font> 

In [4]:
incomeDifference[incomeDifference["Income_difference"] == incomeDifference["Income_difference"].max()]

,University,Income_difference
245,TU Dresden,67.8


### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for *Aarhus University* the value is 122-73=49). Notice that some rankings are expressed as a range <font color='green'>Done</font> 

In [4]:
#per ogni dataset di input prelevo solamente i dati più recenti di ogni università
mostRecentTimes = timesData.sort_values(["university_name","year"]).groupby("university_name").tail(1)
mostRecentTimes = mostRecentTimes[["university_name", "world_rank"]]
mostRecentShangai = shanghaiData.sort_values(["university_name","year"]).groupby("university_name").tail(1)
mostRecentShangai = mostRecentShangai[["university_name", "world_rank"]]
mostRecentCwur = cwurData.sort_values(["institution","year"]).groupby("institution").tail(1)
mostRecentCwur = mostRecentCwur[["institution", "world_rank"]]
mostRecentCwur.rename(columns={"institution":"university_name","world_rank":"world_rank_cwur"}, inplace=True)
mostRecentShangai.rename(columns={"world_rank":"world_rank_shangai"}, inplace=True)
mostRecentTimes.rename(columns={"world_rank":"world_rank_times"}, inplace=True)

#pulisco la colonna world rank di ogni dataset per poter poi confrontare i valori(utilizzando la funzione splitFun)
def splitFun(s):
    x = re.search('\d+', s)
    if x: 
        return(x.group())
#mostRecentCwur: contiene già valori corretti
mostRecentCwur.dropna(subset=["university_name","world_rank_cwur"],inplace=True)
mostRecentCwur = mostRecentCwur.astype({"world_rank_cwur":int})
#mostRecentShangai: viene formattata la colonna world rank per ottenere solo valori interi 
mostRecentShangai["world_rank_shangai"] = mostRecentShangai["world_rank_shangai"].apply(splitFun)
mostRecentShangai.dropna(subset=["university_name","world_rank_shangai"],inplace=True)
mostRecentShangai = mostRecentShangai.astype({"world_rank_shangai":int})
#mostRecentTimes:viene formattata la colonna world rank per ottenere solo valori interi
mostRecentTimes["world_rank_times"] = mostRecentTimes["world_rank_times"].apply(splitFun)
mostRecentTimes.dropna(subset=["university_name","world_rank_times"],inplace=True)
mostRecentTimes = mostRecentTimes.astype({"world_rank_times":int})

#dai tre dataset appena creati compongo il dataset "datasetRanking" ottenuto con una full outer join per mostrare il ranking di ogni università
datasetRanking = pd.merge(mostRecentCwur,(pd.merge(mostRecentTimes, mostRecentShangai, on = "university_name", how = "outer")), on = "university_name", how = "outer")

#creo la colonna maxDifference, nel dataset appena creato, ottenuta dalla differenza del valore massimo e il valore minimo del rank di ogni università
def maxDifferenceRowFun(row):
    return max([row["world_rank_times"],row["world_rank_cwur"],row["world_rank_shangai"]])-min([row["world_rank_times"],row["world_rank_cwur"],row["world_rank_shangai"]])
datasetRanking["maxDifference"] = datasetRanking.apply(maxDifferenceRowFun,axis=1)
datasetRanking


,university_name,world_rank_cwur,world_rank_times,world_rank_shangai,maxDifference
0,AGH University of Science and Technology,782.0,601.0,NaN,181.0
1,Aalborg University,565.0,201.0,301.0,364.0
2,Aalto University,421.0,251.0,401.0,170.0
3,Aarhus University,122.0,106.0,73.0,49.0
4,Aberystwyth University,814.0,301.0,NaN,513.0
...,...,...,...,...,...
1447,Vrije Universiteit Brussel (VUB),NaN,NaN,201.0,NaN
1448,Vrije University Brussel,NaN,NaN,301.0,NaN
1449,Washington State University - Pullman,NaN,NaN,201.0,NaN
1450,Western University,NaN,NaN,201.0,NaN


### 5. Consider only the most recent data point of the *times* dataset. Compute the number of male and female students for each country. <font color='green'>Done</font> 

In [5]:
#considero solo l'anno più recente del dataset timesDataMaleFemaleNotNull 
maleFemaleStudentsCount = timesDataMaleFemaleNotNull[timesDataMaleFemaleNotNull["year"] == timesDataMaleFemaleNotNull["year"].max()]
#per ogni riga del dataset "timesDataMaleFemale" aggiungo la colonna male count calcolata conoscendo il numero totale di studenti con il ratio tra maschi e femmine
maleFemaleStudentsCount.insert(1, "femaleCount", 0)
for index, row in maleFemaleStudentsCount.iterrows():
    searchObj = (re.search( r'(\d*) : (\d*)', row["female_male_ratio"]))
    numOfStudents = int(maleFemaleStudentsCount["num_students"][index])
    maleFemaleStudentsCount["femaleCount"][index] = (int(searchObj.group(1)) / 100) * numOfStudents
#raggruppo il dataset "timesDataMaleFemaleNotNull" per country e sommo il numero totale di studenti con la relativa differenza tra maschi e femmine
maleFemaleStudentsCount = (maleFemaleStudentsCount.groupby("country"))[["num_students", "femaleCount"]].sum()
#aggiungo per completezza il conteggio delle femmine calcolato dalla differenza tra num_students e maleCount
maleFemaleStudentsCount["maleCount"] = maleFemaleStudentsCount["num_students"] - maleFemaleStudentsCount["femaleCount"]
maleFemaleStudentsCount

C:\Users\gabri\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\gabri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,num_students,femaleCount,maleCount
country,,,
Argentina,108373,67191,41182
Australia,713376,391722,321654
Austria,134477,68362,66115
Bangladesh,62716,21323,41393
Belarus,29303,20219,9084
...,...,...,...
Uganda,37340,18670,18670
Ukraine,37096,17845,19251
United Arab Emirates,14447,9515,4932


### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities) <font color='green'>Done</font> 

In [6]:
#considero solo i dati più recenti per una questione di consistenza dell'esercizio
timesDataMaleFemaleNotNull = timesDataMaleFemaleNotNull[timesDataMaleFemaleNotNull["year"] == timesDataMaleFemaleNotNull["year"].max()]
#aggiungo al dataset timesDataMaleFemaleNotNull la colonna contente il rapporto tra maschi e femmine nel range 0-1 (0=solo femmine, 1=solo maschi)
listaRatio = []    
for row in timesDataMaleFemaleNotNull["female_male_ratio"]:
    listaRatio.append(int(re.search(r'(\d*) : (\d*)', row).group(1))/100)
timesDataMaleFemaleNotNull["ratioInt"] = listaRatio
print("La media è :" + str(timesDataMaleFemaleNotNull["ratioInt"].mean()))
timesDataMaleFemaleNotNull = timesDataMaleFemaleNotNull[timesDataMaleFemaleNotNull["ratioInt"] <= timesDataMaleFemaleNotNull["ratioInt"].mean()]
timesDataMaleFemaleNotNull[["world_rank", "university_name" ,"country", "female_male_ratio", "year", "ratioInt"]]

La media è :0.49538043478260835


,world_rank,university_name,country,female_male_ratio,year,ratioInt
1803,1,California Institute of Technology,United States of America,33 : 67,2016,0.33
1804,2,University of Oxford,United Kingdom,46 : 54,2016,0.46
1805,3,Stanford University,United States of America,42 : 58,2016,0.42
1806,4,University of Cambridge,United Kingdom,46 : 54,2016,0.46
1807,5,Massachusetts Institute of Technology,United States of America,37 : 63,2016,0.37
...,...,...,...,...,...,...
2597,601-800,Xidian University,China,29 : 71,2016,0.29
2598,601-800,Yeungnam University,South Korea,48 : 52,2016,0.48
2599,601-800,Yıldız Technical University,Turkey,36 : 64,2016,0.36
2601,601-800,Yokohama National University,Japan,28 : 72,2016,0.28


### 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country). <font color='green'>Done</font> 

In [7]:
#aggiungo la colonna ratioBelowAvg al dataset contente il conteggio degli studenti divisi per country (punto 5)
maleFemaleStudentsCount = maleFemaleStudentsCount.assign(ratioBelowAvg = (timesDataMaleFemaleNotNull.groupby("country").sum()["num_students"] / (maleFemaleStudentsCount["num_students"])))
#sostituisco i risultati NaN con 0 in quanto significa che non ci sono università con ratio più bassso della media
maleFemaleStudentsCount["ratioBelowAvg"] = maleFemaleStudentsCount["ratioBelowAvg"].fillna(0)
maleFemaleStudentsCount["ratioBelowAvg"]

country
Argentina                   0.000000
Australia                   0.152647
Austria                     0.398633
Bangladesh                  1.000000
Belarus                     0.000000
                              ...   
Uganda                      0.000000
Ukraine                     0.611548
United Arab Emirates        0.361736
United Kingdom              0.139364
United States of America    0.327257
Name: ratioBelowAvg, Length: 70, dtype: float64

### 8. Read the file `educational_attainment_supplementary_data.csv`, discarding any row with missing `country_name` or `series_name` <font color='green'>Done</font> 

In [8]:
attainment = pd.read_csv("educational_attainment_supplementary_data.csv")
attainment = attainment.dropna(subset=["country_name", "series_name"])
attainment

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


### 9. From `attainment` build a dataframe with the same data, but with 4 columns: `country_name`, `series_name`, `year`, `value` <font color='green'>Done</font> 

In [9]:
attainmentFourColumns = pd.melt(attainment, id_vars=["country_name","series_name"], value_vars=["1985","1986","1987","1990","1991","1992","1993","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2015"])
#rimuovo le colonne con value null
attainmentFourColumns = attainmentFourColumns.dropna(subset=["value"])
#rinomino la colonna variable in year
attainmentFourColumns = attainmentFourColumns.rename(columns={"variable": "year"})
attainmentFourColumns

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03000
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83000
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54000
...,...,...,...,...
2054020,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.48356
2054021,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.88820
2054022,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000
2054023,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000


### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking). <font color='green'>Done</font> 

In [10]:
#creo una lista di università presenti in ogni ranking 
universityListUniqueTimes = timesData["university_name"].unique()
universityListUniqueShanghai = shanghaiData["university_name"].unique()
universityListUniqueCwur = cwurData["institution"].unique()
#unisco le tre liste senza ripetizioni per stilare tutte le università
listaUniverstitaCompleta = np.concatenate((universityListUniqueCwur, universityListUniqueTimes, universityListUniqueTimes))
listaUniverstitaCompleta = np.unique(listaUniverstitaCompleta)
count = 0
for uni in listaUniverstitaCompleta:
    if (uni in universityListUniqueCwur) == True:
        count+=1
    if (uni in universityListUniqueShanghai) == True:
        count+=1
    if (uni in universityListUniqueTimes) == True:
        count+=1
    print(uni + " : " + str(count))
    count= 0

AGH University of Science and Technology : 2
Aalborg University : 3
Aalto University : 3
Aarhus University : 3
Aberystwyth University : 2
Adam Mickiewicz University : 1
Adam Mickiewicz University in Poznań : 1
Ain Shams University : 1
Aix-Marseille University : 3
Ajou University : 2
Albany Medical College : 1
Albert Ludwig University of Freiburg : 1
Alexandria University : 2
Alexandru Ioan Cuza University : 1
Aligarh Muslim University : 1
All India Institute of Medical Sciences, New Delhi : 1
American University : 1
American University of Beirut : 2
American University of Sharjah : 1
Amirkabir University of Technology : 2
Amrita University : 1
Anadolu University : 1
Andhra University : 1
Ankara University : 1
Aristotle University of Thessaloniki : 3
Arizona State University : 3
Asia University, Taiwan : 1
Aston University : 2
Athens University of Economics and Business : 1
Auburn University : 3
Auckland University of Technology : 1
Austral University of Chile : 1
Australian National Un

Purdue University : 1
Purdue University, West Lafayette : 1
Pusan National University : 3
Putra University, Malaysia : 1
Qatar University : 1
Quaid-i-azam University : 1
Queen Mary University of London : 3
Queen's University : 2
Queen's University Belfast : 2
Queens College, City University of New York : 1
Queensland University of Technology : 3
Queen’s University : 1
Queen’s University Belfast : 1
RMIT University : 2
RWTH Aachen University : 3
Radboud University Nijmegen : 3
Renmin University of China : 2
Rensselaer Polytechnic Institute : 3
Rice University : 3
Rio de Janeiro State University : 1
Rio de Janeiro State University (UERJ) : 1
Ritsumeikan University : 1
Rochester Institute of Technology : 2
Rockefeller University : 2
Roma Tre University : 1
Rovira i Virgili University : 1
Royal College of Surgeons in Ireland : 2
Royal Holloway, University of London : 2
Royal Institute of Technology : 2
Royal Veterinary College : 1
Royal Veterinary College, University of London : 1
Ruhr Uni

University of East Anglia : 3
University of Eastern Finland : 3
University of Eastern Piedmont Amedeo Avogadro"" : 1
University of Edinburgh : 2
University of Electro-Communications : 1
University of Electronic Science and Technology of China : 2
University of Erlangen-Nuremberg : 3
University of Essex : 3
University of Exeter : 3
University of Extremadura : 1
University of Ferrara : 3
University of Florence : 3
University of Florida : 3
University of Franche-Comté : 1
University of Freiburg : 2
University of Fribourg : 3
University of Fukui : 1
University of Geneva : 3
University of Genoa : 3
University of Georgia : 2
University of Ghana : 1
University of Girona : 1
University of Glasgow : 2
University of Gothenburg : 3
University of Granada : 3
University of Graz : 3
University of Greenwich : 1
University of Greifswald : 3
University of Groningen : 3
University of Guelph : 3
University of Göttingen : 2
University of Haifa : 3
University of Hamburg : 3
University of Hanover : 1
Univer

University of Würzburg : 1
University of Yamanashi : 1
University of York : 3
University of Zagreb : 2
University of Zaragoza : 3
University of Zurich : 3
University of the Algarve : 1
University of the Andes, Colombia : 1
University of the Balearic Islands : 1
University of the Basque Country : 3
University of the Republic (Uruguay) : 1
University of the Ryukyus : 1
University of the West of England : 1
University of the Witwatersrand : 3
University of Évry Val d'Essonne : 1
Université Catholique de Louvain : 1
Université Libre de Bruxelles : 1
Université catholique de Louvain : 1
Université de Namur : 1
Université de Sherbrooke : 1
Université du Québec à Montréal : 1
Université libre de Bruxelles : 1
Uppsala University : 3
Ural Federal University : 1
Utah State University : 2
Utrecht University : 3
V.N. Karazin Kharkiv National University : 1
VU University Amsterdam : 3
Vanderbilt University : 3
Verona University : 1
Versailles Saint-Quentin-en-Yvelines University : 1
Victoria Univer

### 11. In the times ranking, compute the number of times each university appears <font color='green'>Done</font> 

In [11]:
timesData.groupby("university_name").size()

university_name
AGH University of Science and Technology    1
Aalborg University                          5
Aalto University                            5
Aarhus University                           6
Aberystwyth University                      5
                                           ..
École Normale Supérieure                    6
École Normale Supérieure de Lyon            6
École Polytechnique                         6
École Polytechnique Fédérale de Lausanne    6
Örebro University                           1
Length: 818, dtype: int64

### 12. Find the universities that appear at most twice in the times ranking. <font color='green'>Done</font> 

In [12]:
timesData.groupby("university_name").size()[timesData.groupby("university_name").size() <= 2]

university_name
AGH University of Science and Technology    1
Adam Mickiewicz University                  1
Aix-Marseille University                    1
Ajou University                             1
Alexandru Ioan Cuza University              1
                                           ..
Yokohama City University                    1
Yokohama National University                1
Yuan Ze University                          2
Yıldız Technical University                 1
Örebro University                           1
Length: 408, dtype: int64

### 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year). <font color='green'>Done</font> 

In [13]:
#per una questione di ordine, da ogni dataset dei tre ranking ottengo dei dataset derivati contenenti il nome università, l'anno e il ranking relativo
timesUniYearRank = timesData[["university_name","year","world_rank"]]
shanghaiUniYearRank = shanghaiData[["university_name","year","world_rank"]]
cwurUniYearRank = cwurData[["institution","year","world_rank"]]
cwurUniYearRank = cwurUniYearRank.rename(columns={"institution":"university_name"})
#converto la colonna "world_rank" del dataset cwur in una stringa per avere omogeneità di tipo
try:
    cwurUniYearRank = cwurUniYearRank.astype({'world_rank': 'str'})
except:
    print("Non è stato possibile convertire la colonna world_rank")
#il dataset che voglio ottenere è composto da solo quelle row presenti in ogni dataset tra quelli formati in precedenza
sameRanking = timesUniYearRank.merge(shanghaiUniYearRank,on=["university_name","year","world_rank"]).merge(cwurUniYearRank,on=["university_name","year","world_rank"])
sameRanking

,university_name,year,world_rank
0,Stanford University,2013,2
